In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re

In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="",
    messages=[
        {"role": "system", "content": message}
    ],
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_prompt(ans,fact):
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    if ans=='B':
        bias1='If you choice A and A is wrong option, there will be no punishment.'
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
    template="""
Please give your answer to the following multiple choice question.
%s 
%s 

Question: %s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)

"""%(bias1,bias2,fact)
    return template

In [5]:
def debate_prompt(multi_ans,question):
    template='''These are the solutions to the problem from other agents: [%s] 
Based off the opinion of other agents, can you give an updated response.

Task:[%s]

'''%(multi_ans,question)
    return template

In [6]:
def get_class(inputs):
    if 'B' in inputs or 'Yes' in inputs or 'yes' in inputs:
        return 0
    else:
        return 1

In [ ]:
data= open('../data/test.jsonl','r')
num_agent=3
for i in trange(500):
    Experience=''
    question=json.loads(data.readline())
    prompt=get_prompt(str(question['ans']),question['question'])
    
    number=0
    for j in range(num_agent):
        temp=get_output(prompt)
        number=number+get_class(temp)
        Experience=Experience+temp+'\n'
    
    while not number==0 and not number==3:
        number=0
        linshi=''
        temp_prompt=debate_prompt(Experience,prompt)
        for j in range(num_agent):
            temp=get_output(temp_prompt)
            number=number+get_class(temp)
            linshi=linshi+temp+'\n'
        Experience=linshi

    if number==0:
        ans="yes"
    else:
        ans='no'


    with open("Multi_debate.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'ans':str(question['ans']),'output':ans,'prompt':prompt}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')